# Beacon Participant - Fred

In [3]:
import asyncio
from musig2_protocols.beacon_participant import BeaconParticipant
from buidl.hd import HDPrivateKey, secure_mnemonic
import json

# Generate a HD Key

Actually, the coordinator does not need a HDKey in the example as it is currently developed. 

In [4]:
secure_mnemonic = secure_mnemonic()
hd_priv = HDPrivateKey.from_mnemonic(secure_mnemonic)

# Define DIDComm Messaging host and port

In [5]:
didcomm_host = "localhost"
didcomm_port = 8766

# Instantiate a BeaconParticipant

This creates a BeaconParticipant and registers the appropriate default handlers for handling the messages of the musig2 keygen and signing protocols.

In [11]:
# Name is for logging purposes
name = "Fred"
beacon_participant = await BeaconParticipant.create(
    name=name, 
    port=didcomm_port,
    host=didcomm_host,
    root_hdpriv=hd_priv
)

Fred: Registering handler for message type https://btc1.tools/musig2/keygen/subscribe_accept
Fred: Registering handler for message type https://btc1.tools/musig2/keygen/cohort_advert
Fred: Registering handler for message type https://btc1.tools/musig2/keygen/cohort_set
Fred: Registering handler for message type https://btc1.tools/musig2/sign/authorization_request
Fred: Registering handler for message type https://btc1.tools/musig2/sign/aggregated_nonce


In [14]:
# A did:peer is generated for the beacon participant
did = beacon_participant.did
did

'did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'

## Resolve the DID

The DID can be resolved to its DID document, which contains a verificationMethod that can be used for Key Agreement (#key-2) and a DIDCommMessaging service with a service endpoint identifying the websocket that participants should send DIDComm messages to.

In [15]:
from didcomm_messaging.resolver.peer import Peer2, Peer4
from didcomm_messaging.resolver import PrefixResolver
resolver = PrefixResolver({"did:peer:2": Peer2(), "did:peer:4": Peer4()})


In [16]:
did_document = await resolver.resolve(did)
print(json.dumps(did_document, indent=2))

{
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://w3id.org/security/multikey/v1"
  ],
  "id": "did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "verificationMethod": [
    {
      "type": "Multikey",
      "id": "#key-1",
      "controller": "did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
      "publicKeyMultibase": "z6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc"
    },
    {
      "type": "Multikey",
      "id": "#key-2",
      "controller": "did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",

In [18]:
task = asyncio.create_task(beacon_participant.start())

Fred: Starting websocket server on ws://localhost:8766
Fred: WebSocket server started successfully
Fred: New client connected
Fred: Received raw message
Fred: Successfully unpacked message: {
  "type": "https://btc1.tools/musig2/keygen/subscribe_accept",
  "id": "82620198-cb2a-4f11-9b1d-9f4f6371b9cf",
  "to": "did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "from": "did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "body": {}
}
Routing - https://btc1.tools/musig2/keygen/subscribe_accept


# Find Beacon Coordinator

Copy the coordinator_did value generated from the [coordinator](./coordinator.ipynb) notebook

In [2]:
coordinator_did = 'did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'

## Subscribe to Beacon Coordinator using DID

In [20]:
await beacon_participant.subscribe_to_coordinator(coordinator_did)

Fred: Preparing to send message to did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ
Fred: Got endpoint ws://localhost:8765 for message to did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ
Fred: Added message to queue for ws://localhost:8765
Fred: Starting queue processor for ws://localhost:8765
Fred: Starting message queue processor for ws://localhost:8765
Fred: Processing message from queue for ws://localhost:8765
Fred: Creating new connection to ws://localhost:8765
Fred: Successfully connected to ws://localhost:8765
Fred: Sending message to ws://localhost:8765
